In [2]:
# This import is required only for jupyter notebooks, since they have their own eventloop
import nest_asyncio
# import browser_use

nest_asyncio.apply()

In [3]:
# !pip3 install "anyio<4"


In [8]:
import os
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
import anthropic
# from google.colab import userdata
import dotenv

dotenv.load_dotenv()

# Get the API key from the environment
# api_key = os.getenv("OPENAI_API_KEY")
# llm = ChatOpenAI(model="gpt-4o", temperature=0)
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, api_key=api_key)

# To use Claude with LangChain:
api_key = os.getenv("ANTHROPIC_API_KEY")
client = anthropic.Anthropic(api_key=api_key)


llm = ChatAnthropic(
    model="claude-3-7-sonnet-20250219",  # Or another Claude model
    temperature=0,
    anthropic_api_key=api_key,
)

In [9]:
# api_key

In [10]:
llm.invoke("hi")

AIMessage(content="Hello! How can I assist you today? Feel free to ask any questions or let me know what you'd like to discuss.", additional_kwargs={}, response_metadata={'id': 'msg_019TmPsQW3vX6k4fAujPS3Lg', 'model': 'claude-3-7-sonnet-20250219', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 8, 'output_tokens': 29}}, id='run-35a5c7d8-0028-40dc-afbe-d1d87dd341f9-0', usage_metadata={'input_tokens': 8, 'output_tokens': 29, 'total_tokens': 37, 'input_token_details': {'cache_read': 0, 'cache_creation': 0}})

In [11]:
from browser_use import Agent, Browser
from browser_use import BrowserConfig
from langchain_openai import ChatOpenAI
import asyncio

# Write the logs to a file download_agent_log.log
import logging

# Set up logging
logging.basicConfig(
    filename="agent_log.log",
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
)
logger = logging.getLogger("browser_agent")


# Basic configuration for the browser
config = BrowserConfig(
    headless= True,  # Run in headless mode True
    # disable_security=True  # Uncomment if you want to disable security
)

# Initialize the browser with the specified configuration
browser = Browser(config=config)

async def main():

    folder_path = "/Users/mac/Documents/projects/investigation/investigations/browser-use/examples/download_and_upload/images"
    # Initialize the agent with the task and language model

    agent = Agent(
        task=f"Please download an image of a dog into images folder. Make sure to not save it in a temportary folder but in {folder_path} as well. It should not be deleted when the browser is closed. If you can't find the download the image stop and return. There might not be a download button.",
        llm=llm,  # Replace with your LLM configuration
        browser=browser,
        generate_gif=False,  # Disable GIF generation
    )

    # Run the agent and get results asynchronously
    result = await agent.run()
    # Process results token-wise
    for action in result.action_results():

        content = action.extracted_content

        print(content, end="\r", flush=True)
        print("\n\n")

        # Log each action result
        logger.info(f"Action result: /n{content}/n")
        # if action.is_done:
        #     print(action.extracted_content)

    # Close the browser after completion
    await browser.close()

# Run the asynchronous main function
asyncio.run(main())

INFO     [agent] 🚀 Starting task: Please download an image of a dog into images folder. Make sure to not save it in a temportary folder but in /Users/mac/Documents/projects/investigation/investigations/browser-use/examples/download_and_upload/images as well. It should not be deleted when the browser is closed. If you can't find the download the image stop and return. There might not be a download button.
INFO     [agent] 📍 Step 1
INFO     [agent] 🤷 Eval: Unknown - This is the first step, so there is no previous goal to evaluate.
INFO     [agent] 🧠 Memory: I need to download an image of a dog and save it to the specific folder: /Users/mac/Documents/projects/investigation/investigations/browser-use/examples/download_and_upload/images. I'm starting with a blank page.
INFO     [agent] 🎯 Next goal: Search for dog images on Google
INFO     [agent] 🛠️  Action 1/1: {"search_google":{"query":"dog images free download"}}
INFO     [controller] 🔍  Searched for "dog images free download" in Google


KeyboardInterrupt: 

In [ ]:
# display(result.action_results())
for action in result.action_results():
  if action.is_done:
    print(action.extracted_content)


LangChain is a composable framework designed for building applications with large language models (LLMs). It simplifies the integration of language models with external data sources and is open-source, supported by an active community. LangChain provides tools for developers to streamline the application lifecycle of LLMs.


## Download

In [12]:
os.path.join(os.path.expanduser("~"),
"Downloads")

#/Users/mac/Downloads

'/Users/mac/Downloads'

In [13]:
# https://github.com/browser-use/browser-use/blob/aefa4670d955cd100794f6b4d69bb637279e93db/examples/features/download_file.py#L24

import asyncio
import os

from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import SecretStr

from browser_use import Agent
from browser_use.browser.browser import Browser, BrowserConfig
from browser_use.browser.context import BrowserContextConfig

load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")
if not api_key:
    raise ValueError("GEMINI_API_KEY is not set")
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp", api_key=SecretStr(api_key))

browser = Browser(
    config=BrowserConfig(
        new_context_config=BrowserContextConfig(
            save_downloads_path=os.path.join(os.path.expanduser("~"), "Downloads")
        )
    )
)


async def run_download():
    agent = Agent(
        task=(
            "Go to https://unsplash.com/s/photos/dogs and download an image of a dog. For image, first hover so you can see the download button (bottom corner). Some images are not free to download, usually they have lock icons in the download button. Only download the free images." 
        ),
        llm=llm,
        max_actions_per_step=8,
        use_vision=True,
        browser=browser,
    )
    await agent.run(max_steps=25)
    await browser.close()


if __name__ == "__main__":
    asyncio.run(run_download())

INFO     [agent] 🚀 Starting task: Go to https://unsplash.com/s/photos/dogs and download an image of a dog. For image, first hover so you can see the download button (bottom corner). Some images are not free to download, usually they have lock icons in the download button. Only download the free images.
INFO     [agent] 📍 Step 1
INFO     [agent] 🤷 Eval: Unknown - I have just started the task
INFO     [agent] 🧠 Memory: I need to go to unsplash and search for dogs
INFO     [agent] 🎯 Next goal: Go to the url https://unsplash.com/s/photos/dogs
INFO     [agent] 🛠️  Action 1/1: {"go_to_url":{"url":"https://unsplash.com/s/photos/dogs"}}
INFO     [controller] 🔗  Navigated to https://unsplash.com/s/photos/dogs
INFO     [agent] 📍 Step 2
INFO     [agent] 👍 Eval: Success - I have loaded the page
INFO     [agent] 🧠 Memory: I need to download an image of a dog from unsplash.com/s/photos/dogs. I need to scroll down to see more images.
INFO     [agent] 🎯 Next goal: Scroll down to see more images
INFO  

# Upload

In [14]:
# os.path.dirname(os.path.dirname(os.path.abspath(__file__)))



In [ ]:
# https://github.com/browser-use/browser-use/blob/aefa4670d955cd100794f6b4d69bb637279e93db/examples/custom-functions/file_upload.py#L32

import os
import sys
from pathlib import Path

from browser_use.agent.views import ActionResult

# sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))

import asyncio
import logging

from langchain_openai import ChatOpenAI

from browser_use import Agent, Controller
from browser_use.browser.browser import Browser, BrowserConfig
from browser_use.browser.context import BrowserContext

logger = logging.getLogger(__name__)

# Initialize controller first
browser = Browser(
	config=BrowserConfig(
		headless=False,
		#browser_instance_path='/Applications/Google Chrome.app/Contents/MacOS/Google Chrome',
	)
)
controller = Controller()


@controller.action(
	'Upload file to interactive element with file path ',
)
async def upload_file(index: int, path: str, browser: BrowserContext, available_file_paths: list[str]):
	if path not in available_file_paths:
		return ActionResult(error=f'File path {path} is not available')

	if not os.path.exists(path):
		return ActionResult(error=f'File {path} does not exist')

	dom_el = await browser.get_dom_element_by_index(index)

	file_upload_dom_el = dom_el.get_file_upload_element()

	if file_upload_dom_el is None:
		msg = f'No file upload element found at index {index}'
		logger.info(msg)
		return ActionResult(error=msg)

	file_upload_el = await browser.get_locate_element(file_upload_dom_el)

	if file_upload_el is None:
		msg = f'No file upload element found at index {index}'
		logger.info(msg)
		return ActionResult(error=msg)

	try:
		await file_upload_el.set_input_files(path)
		msg = f'Successfully uploaded file to index {index}'
		logger.info(msg)
		return ActionResult(extracted_content=msg, include_in_memory=True)
	except Exception as e:
		msg = f'Failed to upload file to index {index}: {str(e)}'
		logger.info(msg)
		return ActionResult(error=msg)


@controller.action('Read the file content of a file given a path')
async def read_file(path: str, available_file_paths: list[str]):
	if path not in available_file_paths:
		return ActionResult(error=f'File path {path} is not available')

	with open(path, 'r') as f:
		content = f.read()
	msg = f'File content: {content}'
	logger.info(msg)
	return ActionResult(extracted_content=msg, include_in_memory=True)


def create_file(file_type: str = 'txt'):
	with open(f'tmp.{file_type}', 'w') as f:
		f.write('test')
	file_path = Path.cwd() / f'tmp.{file_type}'
	logger.info(f'Created file: {file_path}')
	return str(file_path)


async def main():
	task = f'Go to https://kzmpmkh2zfk1ojnpxfn1.lite.vusercontent.net/ and - read the file content and upload them to fields'

	available_file_paths = [create_file('txt'), create_file('pdf'), create_file('csv')]

	model = ChatOpenAI(model='gpt-4o')
	agent = Agent(
		task=task,
		llm=model,
		controller=controller,
		browser=browser,
		available_file_paths=available_file_paths,
	)

	await agent.run()

	await browser.close()

	input('Press Enter to close...')


if __name__ == '__main__':
	asyncio.run(main())

INFO     [__main__] Created file: /Users/mac/Documents/projects/investigation/investigations/browser-use/examples/tmp.txt
INFO     [__main__] Created file: /Users/mac/Documents/projects/investigation/investigations/browser-use/examples/tmp.pdf
INFO     [__main__] Created file: /Users/mac/Documents/projects/investigation/investigations/browser-use/examples/tmp.csv
INFO     [agent] 🚀 Starting task: Go to https://kzmpmkh2zfk1ojnpxfn1.lite.vusercontent.net/ and - read the file content and upload them to fields
INFO     [agent] 📍 Step 1


I0000 00:00:1742840191.859265 1267077 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


INFO     [agent] 🤷 Eval: Unknown - Starting with a blank page.
INFO     [agent] 🧠 Memory: Begin by navigating to the target URL.
INFO     [agent] 🎯 Next goal: Navigate to the given URL.
INFO     [agent] 🛠️  Action 1/1: {"go_to_url":{"url":"https://kzmpmkh2zfk1ojnpxfn1.lite.vusercontent.net/"}}
INFO     [controller] 🔗  Navigated to https://kzmpmkh2zfk1ojnpxfn1.lite.vusercontent.net/
INFO     [agent] 📍 Step 2
INFO     [agent] 👍 Eval: Success - Page with upload options is loaded.
INFO     [agent] 🧠 Memory: Page loaded with different file upload options. Prepare files for upload.
INFO     [agent] 🎯 Next goal: Read and upload content of listed files.
INFO     [agent] 🛠️  Action 1/2: {"read_file":{"path":"/Users/mac/Documents/projects/investigation/investigations/browser-use/examples/tmp.txt"}}
INFO     [agent] 🛠️  Action 2/2: {"upload_file":{"index":0,"path":"/Users/mac/Documents/projects/investigation/investigations/browser-use/examples/tmp.txt"}}
INFO     [__main__] File content: test
INF